# The json thing
Let us face it: json files are pretty much everywhere whenever Open Data lies. One would appreciate working with free and mostly accurate data. One would usually not enjoy the pain of nested files dumped as a json.

Sure, many nice things can be achieved with some efforts, as in this tutorial from [phpflow](https://www.phpflow.com/php/parse-json-data-jquery-datatable/):

![Good looking, right?](img/parse-json-data-jquery.png)

Although when it comes to OpenData, with data coming from [Assemblée Nationale Open Data portal](http://data.assemblee-nationale.fr/static/openData/repository/AMO/tous_acteurs_mandats_organes_xi_legislature/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json.zip), what you get is the following :

In [2]:
more data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json

{"export": {"@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance", "organes"
: {"organe": [{"@xsi:type": "OrganeParlementaire_Type", "uid": "PO428704", "code
Type": "CMP", "libelle": "Commission mixte paritaire charg\u00e9e de proposer un
 texte sur les dispositions restant en discussion de la proposition de loi de li
b\u00e9ralisation des ventes volontaires de meubles aux ench\u00e8res publiques"
, "libelleEdition": "de la proposition de loi de lib\u00e9ralisation des ventes 
volontaires de meubles aux ench\u00e8res publiques", "libelleAbrege": "Ventes au
x ench\u00e8res publiques", "libelleAbrev": "210-07-08", "viMoDe": {"dateDebut":
 "2011-06-01", "dateAgrement": null, "dateFin": "2011-07-25"}, "regime": "5\u00e
8me R\u00e9publique", "legislature": "13", "secretariat": {"secretaire01": null,
 "secretaire02": null}}, {"@xsi:type": "OrganeParlementaire_Type", "uid": "PO428
765", "codeType": "CNPE", "libelle": "Commission d'enqu\u00eate sur les produits
 financiers \u00e0 risque so

Were you proficient in french, you would still not enjoy this sight. Nonetheless, how can this be prettyed?

What about [the pretty tweet](Pretty%20Tweet.ipynb)? Well, it would take way too long. Just thing of Go-sized files. That will not do it.

# jq

This [project](https://github.com/stedolan/jq) is an efficient key to solve this issue. As written in the documentation, jq aims to serve as a sed for json files.

One can install it on its distribution through the relevant packet manager, be it linux, Mac or Cygwin for Windows.

Then, on a shell, you can invoke jq with the following sequence :
__jq 'args' 'filter' 'file'__

## Pretty json

That feat is obtained with the __.__ filter:

In [5]:
jq . data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json | more

{
  "export": {
    "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "organes": {
      "organe": [
        {
          "@xsi:type": "OrganeParlementaire_Type",
          "uid": "PO428704",
          "codeType": "CMP",
          "libelle": "Commission mixte paritaire chargée de proposer un texte su
r les dispositions restant en discussion de la proposition de loi de libéralisat
ion des ventes volontaires de meubles aux enchères publiques",
          "libelleEdition": "de la proposition de loi de libéralisation des vent
es volontaires de meubles aux enchères publiques",
          "libelleAbrege": "Ventes aux enchères publiques",
          "libelleAbrev": "210-07-08",
          "viMoDe": {
            "dateDebut": "2011-06-01",
            "dateAgrement": null,
            "dateFin": "2011-07-25"
          },
          "regime": "5ème République",
          "legislature": "13",



Well done. It took a relative short amount of time and it does work. Note that jq will fail if the json file is not a valid json. In that case, you can use _-f_ argument to force jq, but the propper approach would still to clean the file.

As one would guess, the filter __.featureName__ will display the insides of said feature. What if the json is not a flat file? What about multiple layers?

## Piping filters

OK, the file is nice. That's great. Apparently, it has an _export / organes /organe[]_ sequence. How to get there? Just piping with __|__ the filters until the destination is reached. Let's have a look at the fifth _organe_:

In [ ]:
jq '.export | .organes | .organe | .[4]' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json

Basically, what we did was piping successive requests, from the outer layer of json to the inner. It can translate into _get me the fifth element of the organe table that is in organes object, itself contained in export object_.

Well, to access the _n-th_ of an arry in jq, juste type __.[n-1]__. 

Do note that piping through properties can be written with the following syntax as well:

In [8]:
jq '.export.organes.organe[4]' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json

{
  "@xsi:type": "OrganeParlementaire_Type",
  "uid": "PO428784",
  "codeType": "CMP",
  "libelle": "Commission mixte paritaire chargée de proposer un texte sur les dispositions restant en discussion du projet de loi relatif à la bioéthique",
  "libelleEdition": "du projet de loi relatif à la bioéthique",
  "libelleAbrege": "bioéthique",
  "libelleAbrev": "2911",
  "viMoDe": {
    "dateDebut": "2011-06-10",
    "dateAgrement": null,
    "dateFin": "2011-07-25"
  },
  "regime": "5ème République",
  "legislature": "13",
  "secretariat": {
    "secretaire01": null,
    "secretaire02": null
  }
}


You might want the latter syntax for it is shorter, and maybe more understandable if you come from object -oriented programming.

## Playing with arrays!

Let's say that the previous is not enough, and that we want to get the abridged names for every _organe_. This can be achieved by lefting the__[]__ empty.

In [12]:
jq '.export.organes.organe[].libelleAbrege' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json | more

"Ventes aux enchères publiques"
"Commission d'enquête Produits financiers à risque"
"Commission d'enquête Financement syndicats"
"Gaz de schiste"
"bioéthique"
"Accidents circulation"
"Loi de finances rectificative pour 2011"
"Immigration"
"Autisme"
"Langue des signes"
"Famille"
"Commerce extérieur"
"Alternance"
"Justice pénale"
"Collectivités article 73 C"
"Français de l'étranger"
"Polynésie française"
"Transports"
"Budget, comptes publics et réforme de l'État"
"Fonction publique"
"Défense et anciens combattants (secrétariat d'État)"
"Guyane-Martinique"
"Français de l'étranger"


Interesting. Just to play with array selector, let's take the same names between the organs 36 and 43. The function __range(35-42)__ will do the trick:

In [17]:
jq '.export.organes.organe[range(35,42)].libelleAbrege' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json | more

"Ventes aux enchères publiques"
"Commission d'enquête Produits financiers à risque"
"Commission d'enquête Financement syndicats"
"Gaz de schiste"
"bioéthique"
"Accidents circulation"
"Loi de finances rectificative pour 2011"
"Immigration"
"Autisme"
"Langue des signes"
"Famille"
"Commerce extérieur"
"Alternance"
"Justice pénale"
"Collectivités article 73 C"
"Français de l'étranger"
"Polynésie française"
"Transports"
"Budget, comptes publics et réforme de l'État"
"Fonction publique"
"Défense et anciens combattants (secrétariat d'État)"
"Guyane-Martinique"
"Français de l'étranger"


Where an other syntax within the brackets would lead to only select:

In [20]:
jq '.export.organes.organe[35,42,43].libelleAbrege' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json | more

"Délégation sénatoriale à l'Outre-mer"
"Commémoration 11 novembre"
"Ports d'outre-mer"


## Erasing some parts
Some times, it might be convenient to remove some irrelevant features or object. This is made possible by the __del()__ function. 
For instance, with there is more than one category of object aside __organes__?

In [21]:
jq '.export | del(.organes)' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json | more

{
  "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
  "acteurs": {
    "acteur": [
      {
        "uid": {
          "@xsi:type": "IdActeur_type",
          "#text": "PA1300"
        },
        "etatCivil": {
          "ident": {
            "civ": "M.",
            "prenom": "Jean-Michel",
            "nom": "Ferrand",
            "alpha": "Ferrand",
            "trigramme": null
          },
          "infoNaissance": {
            "dateNais": "1942-08-31",
            "villeNais": "Gardanne",
            "depNais": "Bouches-du-Rhône",
            "paysNais": null
          },



Basically, the parameter for the __del()__ function is the path you want to erase. It could be also some elements of a given array with a syntax like __del(.array[x])__.
# Exports

## To Json
Unlike what one would argue, this trick allows to make smaller json, easyer to process. For instance, let's assume we want to work with only the people _actors_. 

In [2]:
jq '.export | del(.organes) | .acteurs' data/AMO30_tous_acteurs_tous_mandats_tous_organes_historique.json > data/acteurs.json

{
  "acteur": [
    {
      "uid": {
        "@xsi:type": "IdActeur_type",
        "#text": "PA1300"
      },
      "etatCivil": {
        "ident": {
          "civ": "M.",


In [3]:
head -15 data/acteurs.json

{
  "acteur": [
    {
      "uid": {
        "@xsi:type": "IdActeur_type",
        "#text": "PA1300"
      },
      "etatCivil": {
        "ident": {
          "civ": "M.",
          "prenom": "Jean-Michel",
          "nom": "Ferrand",
          "alpha": "Ferrand",
          "trigramme": null
        },


## To csv
This is usually the end-goal to this kind of manipulation. 
* Generate a table;
* Pipe it through the __@csv__ filter;
* Save it as a csv file.

In [24]:
jq -r '.acteur[] | [.etatCivil.ident.prenom,.etatCivil.ident.nom] | @csv' data/acteurs.json > data/acteurs.csv

In [25]:
head -15 data/acteurs.csv

"Jean-Michel","Ferrand"
"Alain","Ferry"
"Daniel","Fidelin"
"Jean-Yves","Besselat"
"Simon","Renucci"
"Jean-Marie","Rolland"
"Hervé","Novelli"
"Vincent","Descoeur"
"Bernard","Depierre"
"Marie-Françoise","Perol-Dumont"
"Louis","Aliot"
"Frédérique","Tuffnell"
"Éric","Poulliat"
"Florence","Lasserre-David"
"Olivia","Gregoire"


The next step? Well, you got yourself a csv file. Congrats, the hardest has yet to come: clean your data with pandas in python or sed/vi. Well, that part is usually more explained.